# Import packages

In [41]:

import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import pandas as pd
import matplotlib
matplotlib.use(u'nbAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from tqdm import tqdm
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib 
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

# Loading processing and normalize data

In [3]:
df = pd.read_csv("sample_data_intw.csv") 

In [4]:
df.head()

,Unnamed: 0,label,msisdn,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,...,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90,pcircle,pdate
0,1,0,21408I70789,272.0,3055.050000,3065.150000,220.13,260.13,2.0,0.0,...,6.0,0.0,2.0,12,6,0.0,29.000000,29.000000,UPW,2016-07-20
1,2,1,76462I70374,712.0,12122.000000,12124.750000,3691.26,3691.26,20.0,0.0,...,12.0,0.0,1.0,12,12,0.0,0.000000,0.000000,UPW,2016-08-10
2,3,1,17943I70372,535.0,1398.000000,1398.000000,900.13,900.13,3.0,0.0,...,6.0,0.0,1.0,6,6,0.0,0.000000,0.000000,UPW,2016-08-19
3,4,1,55773I70781,241.0,21.228000,21.228000,159.42,159.42,41.0,0.0,...,6.0,0.0,2.0,12,6,0.0,0.000000,0.000000,UPW,2016-06-06
4,5,1,03813I82730,947.0,150.619333,150.619333,1098.90,1098.90,4.0,0.0,...,6.0,0.0,7.0,42,6,0.0,2.333333,2.333333,UPW,2016-06-22


In [87]:
set(df['pcircle'].values)

{'UPW'}

##### Drop pcircle since its just having single value

In [8]:
print("Number of data points in data", df.shape)
print('-'*50)
print("The attributes of data :", df.columns.values)


Number of data points in data (209593, 37)
--------------------------------------------------
The attributes of data : ['Unnamed: 0' 'label' 'msisdn' 'aon' 'daily_decr30' 'daily_decr90'
 'rental30' 'rental90' 'last_rech_date_ma' 'last_rech_date_da'
 'last_rech_amt_ma' 'cnt_ma_rech30' 'fr_ma_rech30' 'sumamnt_ma_rech30'
 'medianamnt_ma_rech30' 'medianmarechprebal30' 'cnt_ma_rech90'
 'fr_ma_rech90' 'sumamnt_ma_rech90' 'medianamnt_ma_rech90'
 'medianmarechprebal90' 'cnt_da_rech30' 'fr_da_rech30' 'cnt_da_rech90'
 'fr_da_rech90' 'cnt_loans30' 'amnt_loans30' 'maxamnt_loans30'
 'medianamnt_loans30' 'cnt_loans90' 'amnt_loans90' 'maxamnt_loans90'
 'medianamnt_loans90' 'payback30' 'payback90' 'pcircle' 'pdate']


In [18]:
count = 0
for val in df['label'].values:
    if val == 1:
        count = count + 1
print("No of class 1 points",count)
print("No of class 0 points", df.shape[0] - count)



No of class 1 points 183431
No of class 0 points 26162


In [27]:
print("No of unique values in date column")
len(set(df['pdate'].values))

No of unique values in date column


82

In [28]:
print("No of unique values in mobile number column")
len(set(df['msisdn'].values))

No of unique values in mobile number column


186243

#### My gut feel was date and phone number were not useful but no of unique dates are quite less than phone number. Therefore drop phone number.

In [5]:
df.drop(['Unnamed: 0','msisdn','pcircle'], axis = 1, inplace = True) 

In [90]:
df.head()

,label,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,cnt_ma_rech30,...,amnt_loans30,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90,pdate
0,0,272.0,3055.050000,3065.150000,220.13,260.13,2.0,0.0,1539,2,...,12,6.0,0.0,2.0,12,6,0.0,29.000000,29.000000,2016-07-20
1,1,712.0,12122.000000,12124.750000,3691.26,3691.26,20.0,0.0,5787,1,...,12,12.0,0.0,1.0,12,12,0.0,0.000000,0.000000,2016-08-10
2,1,535.0,1398.000000,1398.000000,900.13,900.13,3.0,0.0,1539,1,...,6,6.0,0.0,1.0,6,6,0.0,0.000000,0.000000,2016-08-19
3,1,241.0,21.228000,21.228000,159.42,159.42,41.0,0.0,947,0,...,12,6.0,0.0,2.0,12,6,0.0,0.000000,0.000000,2016-06-06
4,1,947.0,150.619333,150.619333,1098.90,1098.90,4.0,0.0,2309,7,...,42,6.0,0.0,7.0,42,6,0.0,2.333333,2.333333,2016-06-22


In [91]:
# Select duplicate rows 
duplicateRowsDF = df[df.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF,duplicateRowsDF.shape)

Duplicate Rows except first occurrence based on all columns are :
        label    aon  daily_decr30  daily_decr90  rental30  rental90  \
35769       0  118.0      0.000000      0.000000      0.00      0.00   
49751       0  347.0     -2.666667     -2.666667      0.00      0.00   
51852       0  671.0      0.000000      0.000000      0.00      0.00   
57581       0  152.0    500.000000    500.000000      0.00      0.00   
68296       0   51.0      0.000000      0.000000      0.00      0.00   
90887       0   81.0      0.000000      0.000000      0.00      0.00   
94430       0  127.0      0.000000      0.000000      0.00      0.00   
98092       0   62.0      0.000000      0.000000      0.00      0.00   
111140      0  150.0    500.000000    500.000000      0.00      0.00   
124619      0  542.0      0.000000      0.000000      0.00      0.00   
135203      0   53.0      0.000000      0.000000      0.00      0.00   
137416      0  339.0      0.000000      0.000000      0.00      0.00  

In [6]:
#Dropping duplicates
df.drop_duplicates(inplace = True) 
#Status of duplicates after dropping
duplicateRowsDF = df[df.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF,duplicateRowsDF.shape)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [label, aon, daily_decr30, daily_decr90, rental30, rental90, last_rech_date_ma, last_rech_date_da, last_rech_amt_ma, cnt_ma_rech30, fr_ma_rech30, sumamnt_ma_rech30, medianamnt_ma_rech30, medianmarechprebal30, cnt_ma_rech90, fr_ma_rech90, sumamnt_ma_rech90, medianamnt_ma_rech90, medianmarechprebal90, cnt_da_rech30, fr_da_rech30, cnt_da_rech90, fr_da_rech90, cnt_loans30, amnt_loans30, maxamnt_loans30, medianamnt_loans30, cnt_loans90, amnt_loans90, maxamnt_loans90, medianamnt_loans90, payback30, payback90, pdate]
Index: []

[0 rows x 34 columns] (0, 34)


In [7]:
#https://stackoverflow.com/questions/27159189/find-empty-or-nan-entry-in-pandas-dataframe#:~:text=Check%20if%20the%20columns%20contain,for%20rows%20with%20missing%20data.
np.where(pd.isnull(df)) 

(array([], dtype=int64), array([], dtype=int64))

###### From above its clear that their are no NAN none values

In [8]:
dd = []
mm = []
yy = []
for i in range(df.shape[0]):
    yy.append(int(df['pdate'].values[i].split('-')[0]))
    mm.append(int(df['pdate'].values[i].split('-')[1]))
    dd.append(int(df['pdate'].values[i].split('-')[2]))
    

In [9]:
len(dd),len(mm),len(yy),df.shape[0]

(209562, 209562, 209562, 209562)

In [10]:
df.drop(['pdate'], axis = 1, inplace = True) 

In [11]:
df['Day'] = dd
df['Month'] = mm
df['Year'] = yy
df.head()

,label,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,cnt_ma_rech30,...,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90,Day,Month,Year
0,0,272.0,3055.050000,3065.150000,220.13,260.13,2.0,0.0,1539,2,...,0.0,2.0,12,6,0.0,29.000000,29.000000,20,7,2016
1,1,712.0,12122.000000,12124.750000,3691.26,3691.26,20.0,0.0,5787,1,...,0.0,1.0,12,12,0.0,0.000000,0.000000,10,8,2016
2,1,535.0,1398.000000,1398.000000,900.13,900.13,3.0,0.0,1539,1,...,0.0,1.0,6,6,0.0,0.000000,0.000000,19,8,2016
3,1,241.0,21.228000,21.228000,159.42,159.42,41.0,0.0,947,0,...,0.0,2.0,12,6,0.0,0.000000,0.000000,6,6,2016
4,1,947.0,150.619333,150.619333,1098.90,1098.90,4.0,0.0,2309,7,...,0.0,7.0,42,6,0.0,2.333333,2.333333,22,6,2016


In [12]:
X = df.drop(['label'], axis = 1) 

Y = df.loc[:,'label']

In [13]:
X.head()

,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,cnt_ma_rech30,fr_ma_rech30,...,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90,Day,Month,Year
0,272.0,3055.050000,3065.150000,220.13,260.13,2.0,0.0,1539,2,21.0,...,0.0,2.0,12,6,0.0,29.000000,29.000000,20,7,2016
1,712.0,12122.000000,12124.750000,3691.26,3691.26,20.0,0.0,5787,1,0.0,...,0.0,1.0,12,12,0.0,0.000000,0.000000,10,8,2016
2,535.0,1398.000000,1398.000000,900.13,900.13,3.0,0.0,1539,1,0.0,...,0.0,1.0,6,6,0.0,0.000000,0.000000,19,8,2016
3,241.0,21.228000,21.228000,159.42,159.42,41.0,0.0,947,0,0.0,...,0.0,2.0,12,6,0.0,0.000000,0.000000,6,6,2016
4,947.0,150.619333,150.619333,1098.90,1098.90,4.0,0.0,2309,7,2.0,...,0.0,7.0,42,6,0.0,2.333333,2.333333,22,6,2016


In [14]:
Y.head(5)

0    0
1    1
2    1
3    1
4    1
Name: label, dtype: int64

In [15]:
X.shape,Y.shape

((209562, 35), (209562,))

In [16]:
X_train, X_test, y_train, y_test=train_test_split(X, Y, test_size=0.3, stratify=Y)
X_train, X_cv, y_train, y_cv=train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

In [17]:
print('Train shape:', X_train.shape)
print('Cross validate shape:',X_cv.shape )
print('test shape:',X_test.shape )

Train shape: (117354, 35)
Cross validate shape: (29339, 35)
test shape: (62869, 35)


In [18]:
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(x_scaled)

In [19]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.000518,0.010412,0.008673,0.108854,0.112360,0.000031,0.000027,0.014055,0.039409,0.000002,...,0.0,0.001402,0.102941,0.5,0.0,0.01516,0.015549,0.066667,0.5,0.0
1,0.001905,0.021250,0.022273,0.108437,0.113881,0.000037,0.000027,0.027982,0.009852,0.000007,...,0.0,0.001001,0.073529,0.5,0.0,0.00000,0.033528,0.033333,1.0,0.0
2,0.002008,0.000349,0.000290,0.106606,0.109933,0.000029,0.000027,0.000000,0.000000,0.000000,...,0.0,0.000200,0.014706,0.5,0.0,0.00000,0.000000,0.266667,0.0,0.0
3,0.000232,0.023604,0.019578,0.126258,0.145054,0.000040,0.000027,0.073600,0.009852,0.000000,...,0.0,0.000200,0.014706,0.5,0.0,0.00000,0.000000,0.966667,0.5,0.0
4,0.000930,0.008545,0.010497,0.125078,0.127536,0.000035,0.000027,0.027982,0.004926,0.000000,...,0.0,0.000401,0.029412,0.5,0.0,0.00000,0.221574,0.066667,1.0,0.0


In [20]:
x_scaled = scaler.transform(X_cv)
X_cv = pd.DataFrame(x_scaled)

In [21]:

x_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(x_scaled)

# The loss which I choose to minimize is log-loss I will give below justifications why log-loss?
1. Dataset is imbalanced.
2. If class label is say '1', then we want our model P(Y=1|x) should be close enough to '1'in other words we need to penalize probability scores also and logloss excatly does it. 

# Building maching learning models

# K-NN

In [35]:
alpha = [x for x in range(1, 112,10)]
cv_log_error_array=[]
for i in tqdm(alpha):
    k_cfl=KNeighborsClassifier(n_neighbors=i,n_jobs = -1)
    k_cfl.fit(X_train,y_train)
    sig_clf = CalibratedClassifierCV(k_cfl, method="sigmoid")
    sig_clf.fit(X_train, y_train)
    predict_y = sig_clf.predict_proba(X_cv)
    cv_log_error_array.append(log_loss(y_cv, predict_y, labels=k_cfl.classes_, eps=1e-15))
    



100%|███████████████████████████████████████████████████████████████████████████████| 12/12 [3:12:16<00:00, 961.34s/it]


In [36]:
for i in range(len(cv_log_error_array)):
    print ('log_loss for k = ',alpha[i],'is',cv_log_error_array[i])

best_alpha = np.argmin(cv_log_error_array)
    
print("Best value of K=",alpha[best_alpha])

log_loss for k =  1 is 0.3237706293042584
log_loss for k =  11 is 0.27792200315312576
log_loss for k =  21 is 0.2757627996125274
log_loss for k =  31 is 0.2748891528861179
log_loss for k =  41 is 0.2744119912269251
log_loss for k =  51 is 0.27390889567316623
log_loss for k =  61 is 0.27375227195599944
log_loss for k =  71 is 0.27384826143413415
log_loss for k =  81 is 0.27417219137068133
log_loss for k =  91 is 0.27452009874898187
log_loss for k =  101 is 0.2749181611137194
log_loss for k =  111 is 0.2750130490208926
Best value of K= 61


In [37]:
k_cfl=KNeighborsClassifier(n_neighbors=alpha[best_alpha],n_jobs = -1)
k_cfl.fit(X_train,y_train)
sig_clf = CalibratedClassifierCV(k_cfl, method="sigmoid")
sig_clf.fit(X_train, y_train)
pred_y=sig_clf.predict(X_test)


predict_y = sig_clf.predict_proba(X_train)
print ('log loss for train data',log_loss(y_train, predict_y))
predict_y = sig_clf.predict_proba(X_cv)
print ('log loss for cv data',log_loss(y_cv, predict_y))
predict_y = sig_clf.predict_proba(X_test)
print ('log loss for test data',log_loss(y_test, predict_y))

log loss for train data 0.2664619518152901
log loss for cv data 0.27375227195599944
log loss for test data 0.2713029260674443


In [38]:
# Save the model as a pickle in a file 
joblib.dump(sig_clf, 'KNN.pkl') 
  
# Load the model from the file 
#knn_from_joblib = joblib.load('filename.pkl') 

['KNN.pkl']

# LogisticRegression

In [22]:
alpha = [10 ** x for x in range(-5, 4)]
cv_log_error_array=[]
for i in tqdm(alpha):
    logisticR=LogisticRegression(penalty='l2',C=i,class_weight='balanced',n_jobs = -1)
    logisticR.fit(X_train,y_train)
    sig_clf = CalibratedClassifierCV(logisticR, method="sigmoid")
    sig_clf.fit(X_train, y_train)
    predict_y = sig_clf.predict_proba(X_cv)
    cv_log_error_array.append(log_loss(y_cv, predict_y, labels=logisticR.classes_, eps=1e-15))
    


100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [19:13<00:00, 128.11s/it]


In [23]:
for i in range(len(cv_log_error_array)):
    print ('log_loss for c = ',alpha[i],'is',cv_log_error_array[i])

best_alpha = np.argmin(cv_log_error_array)
    




log_loss for c =  1e-05 is 0.3567824833928973
log_loss for c =  0.0001 is 0.3556054507677887
log_loss for c =  0.001 is 0.3477645314426088
log_loss for c =  0.01 is 0.32310193309676216
log_loss for c =  0.1 is 0.30518750961663155
log_loss for c =  1 is 0.2980549089802739
log_loss for c =  10 is 0.29849799229458107
log_loss for c =  100 is 0.29826560709052125
log_loss for c =  1000 is 0.29919297958415964


In [25]:
print("best value of C:",alpha[best_alpha])
logisticR=LogisticRegression(penalty='l2',C=alpha[best_alpha],class_weight='balanced',n_jobs = -1)
logisticR.fit(X_train,y_train)
sig_clf = CalibratedClassifierCV(logisticR, method="sigmoid")
sig_clf.fit(X_train, y_train)
pred_y=sig_clf.predict(X_test)

predict_y = sig_clf.predict_proba(X_train)
print ('log loss for train data',log_loss(y_train, predict_y, labels=logisticR.classes_, eps=1e-15))
predict_y = sig_clf.predict_proba(X_cv)
print ('log loss for cv data',log_loss(y_cv, predict_y, labels=logisticR.classes_, eps=1e-15))
predict_y = sig_clf.predict_proba(X_test)
print ('log loss for test data',log_loss(y_test, predict_y, labels=logisticR.classes_, eps=1e-15))

best value of C: 1
log loss for train data 0.299492604550692
log loss for cv data 0.2980549089802739
log loss for test data 0.2967357484373374


In [27]:
# Save the model as a pickle in a file 
joblib.dump(sig_clf, 'LogisticReg.pkl') 
  
# Load the model from the file 
#knn_from_joblib = joblib.load('filename.pkl') 

['LogisticReg.pkl']

# RandomForest

In [30]:
alpha=[10,50,100,500,1000,2000,3000]
cv_log_error_array=[]
train_log_error_array=[]
for i in tqdm(alpha):
    r_cfl=RandomForestClassifier(n_estimators=i,n_jobs=-1)
    r_cfl.fit(X_train,y_train)
    sig_clf = CalibratedClassifierCV(r_cfl, method="sigmoid")
    sig_clf.fit(X_train, y_train)
    predict_y = sig_clf.predict_proba(X_cv)
    cv_log_error_array.append(log_loss(y_cv, predict_y, labels=r_cfl.classes_, eps=1e-15))



100%|████████████████████████████████████████████████████████████████████████████████| 7/7 [5:53:33<00:00, 3030.47s/it]


In [31]:
for i in range(len(cv_log_error_array)):
    print ('log_loss for c = ',alpha[i],'is',cv_log_error_array[i])


best_alpha = np.argmin(cv_log_error_array)
print("best value of no of estimators:",alpha[best_alpha])


log_loss for c =  10 is 0.2182014315616614
log_loss for c =  50 is 0.21530736355676566
log_loss for c =  100 is 0.2156391154226347
log_loss for c =  500 is 0.21523965646775692
log_loss for c =  1000 is 0.2152805886413863
log_loss for c =  2000 is 0.2153108934067022
log_loss for c =  3000 is 0.21533023833484888
best value of no of estimators: 500


In [32]:
r_cfl=RandomForestClassifier(n_estimators=alpha[best_alpha],n_jobs=-1)
r_cfl.fit(X_train,y_train)
sig_clf = CalibratedClassifierCV(r_cfl, method="sigmoid")
sig_clf.fit(X_train, y_train)

predict_y = sig_clf.predict_proba(X_train)
print('For values of best alpha = ', alpha[best_alpha], "The train log loss is:",log_loss(y_train, predict_y))
predict_y = sig_clf.predict_proba(X_cv)
print('For values of best alpha = ', alpha[best_alpha], "The cross validation log loss is:",log_loss(y_cv, predict_y))
predict_y = sig_clf.predict_proba(X_test)
print('For values of best alpha = ', alpha[best_alpha], "The test log loss is:",log_loss(y_test, predict_y))

For values of best alpha =  500 The train log loss is: 0.06250020350570407
For values of best alpha =  500 The cross validation log loss is: 0.21525941810920668
For values of best alpha =  500 The test log loss is: 0.21035609653243953


In [33]:
# Save the model as a pickle in a file 
joblib.dump(sig_clf, 'RandomForest.pkl') 
  
# Load the model from the file 
#knn_from_joblib = joblib.load('filename.pkl') 

['RandomForest.pkl']

# Boosting

In [ ]:
    r_cfl=RandomForestClassifier(n_estimators=i,n_jobs=-1)
    r_cfl.fit(X_train,y_train)
    sig_clf = CalibratedClassifierCV(r_cfl, method="sigmoid")
    sig_clf.fit(X_train, y_train)
    predict_y = sig_clf.predict_proba(X_cv)
    cv_log_error_array.append(log_loss(y_cv, predict_y, labels=r_cfl.classes_, eps=1e-15))


In [42]:
X = []
Y = []
for max_depth in  tqdm([2, 5, 10 ,15,20,25]):
    for n_estimators in [5, 15,50,100,500,1000,2000,3000]:
        X.append(max_depth)
        Y.append(n_estimators)
        r_cfl =  lgb.LGBMClassifier(max_depth = max_depth,n_estimators = n_estimators,n_jobs = -1 )
        r_cfl.fit(X_train, y_train)
        sig_clf = CalibratedClassifierCV(r_cfl, method="sigmoid")
        sig_clf.fit(X_train, y_train)
        predict_y = sig_clf.predict_proba(X_cv)
        cv_log_error_array.append(log_loss(y_cv, predict_y, labels=r_cfl.classes_, eps=1e-15))


100%|████████████████████████████████████████████████████████████████████████████████| 6/6 [1:55:23<00:00, 1153.89s/it]


In [43]:
index = np.argmin(cv_log_error_array)
print("BEST PARAM FOR WHICH Log loss min is:" )
print("Max Depth",X[index],"n_estimators",Y[index],"Test score",cv_log_error_array[index])

BEST PARAM FOR WHICH Log loss min is:
Max Depth 15 n_estimators 500 Test score 0.20697170106770077


In [45]:
x_cfl=lgb.LGBMClassifier(max_depth = X[index],n_estimators = Y[index],n_jobs = -1 )
x_cfl.fit(X_train,y_train)
sig_clf = CalibratedClassifierCV(x_cfl, method="sigmoid")
sig_clf.fit(X_train, y_train)
    


In [46]:
predict_y = sig_clf.predict_proba(X_train)
print ("The train log loss is:",log_loss(y_train, predict_y))
predict_y = sig_clf.predict_proba(X_cv)
print("The cross validation log loss is:",log_loss(y_cv, predict_y))
predict_y = sig_clf.predict_proba(X_test)
print( "The test log loss is:",log_loss(y_test, predict_y))

The train log loss is: 0.13883641817942305
The cross validation log loss is: 0.20697170106770077
The test log loss is: 0.2030241507440118


In [47]:
# Save the model as a pickle in a file 
joblib.dump(sig_clf, 'Boosting.pkl') 
  
# Load the model from the file 
#knn_from_joblib = joblib.load('filename.pkl') 

['Boosting.pkl']

# Summary

In [49]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["Best Model", "Train log loss", "Cross validation log loss", "Test log loss"]

x.add_row(["K nearest neighbour",0.26,0.27,0.27 ])
x.add_row(["Logistic regression",0.29,0.29,0.29 ])
x.add_row(["Random forest",0.06,0.21,0.21 ])
x.add_row(["Boosting",0.13,0.20,0.20 ])
print(x)

+---------------------+----------------+---------------------------+---------------+
|      Best Model     | Train log loss | Cross validation log loss | Test log loss |
+---------------------+----------------+---------------------------+---------------+
| K nearest neighbour |      0.26      |            0.27           |      0.27     |
| Logistic regression |      0.29      |            0.29           |      0.29     |
|    Random forest    |      0.06      |            0.21           |      0.21     |
|       Boosting      |      0.13      |            0.2            |      0.2      |
+---------------------+----------------+---------------------------+---------------+


### From above "Boosting" seems to work the best and also notice random forest is over fitting.

# Generate P(yi=1|xi) for each of the test data and storing it in result file

In [54]:
predict_y.shape,y_test.shape,type(predict_y),type(predict_y)

((62869, 2), (62869,), numpy.ndarray, numpy.ndarray)

In [67]:
 data = {'yi':y_test.values, 'P(yi = 1| xi)':predict_y[:,1]} 
  
# Creates pandas DataFrame. 
df = pd.DataFrame(data) 
df.head()

,yi,P(yi = 1| xi)
0,1,0.970815
1,1,0.970815
2,1,0.970805
3,1,0.970708
4,1,0.951456


In [68]:
df.to_csv('result.csv') 